In [17]:
#%pip install --upgrade git+https://github.com/OpenConjecture/pyorient.git

  Cloning https://github.com/OpenConjecture/pyorient.git to /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-req-build-9vbovzwt
  Running command git clone --filter=blob:none --quiet https://github.com/OpenConjecture/pyorient.git /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-req-build-9vbovzwt
  Resolved https://github.com/OpenConjecture/pyorient.git to commit c41c3b615fad09ea9dc71b802339c865d5cdce14
  Preparing metadata (setup.py) ... done
  Created wheel for pyorient: filename=pyorient-1.5.5-py3-none-any.whl size=83239 sha256=3c4501c718b74297d17d31b0213352956515cee15a73ffc1f2a78d4cbc34a4c5
  Stored in directory: /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-ephem-wheel-cache-2z4t75rr/wheels/3f/dd/1c/7382ed6d7620fea508cd831fada5b14f804ed0fb5cdaa89b34
Successfully built pyorient
Note: you may need to restart the kernel to use updated packages.


In [1]:
#%pip install --upgrade git+https://github.com/OpenConjecture/pyorient.git

import pandas as pd
import pyorient #La libreria está muy vieja por lo cual puede tener errores, no se actualiza desde hace 6 años 

In [2]:
#Nos conectamos al servidor/localhost

client = pyorient.OrientDB("localhost", 2424)
client.set_session_token( True )  # set true to enable the token based
client.db_open( "Aero", "root", "root" )

In [3]:
#Importamos las bases de datos, igualmente a como lo hicimos en ArangoDB

aeropuerto_info = pd.read_csv("DataFrames/Info_Basica_Aeropuertos.csv").iloc[:,1:]
arrivos = pd.read_csv("DataFrames/arrivos.csv").iloc[:,1:]
aerolineas = pd.read_csv("DataFrames/Aerolineas.csv").iloc[:,1:]
aerolineas.fillna("NO",inplace=True)
aerolineas.drop_duplicates(subset=['IATA'],inplace=True)
aerolineas = aerolineas[aerolineas['IATA'].str.len() <= 2]
arrivos = arrivos.applymap(str)
arrivos.drop_duplicates(inplace=True,subset=['Numero de Vuelo'])
arrivos["cod origen"] = arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)
arrivos['cod destino'] = arrivos.apply(lambda x: x['Destino'].split()[-1].replace('(','').replace(')',''), axis=1)
aeropuerto_info['cod aero'] = aeropuerto_info.apply(lambda x: x['Nombre'].split()[-1].replace('(','').replace(')',''), axis=1)
arrivos['cod vuelo'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2]+a[3:])
arrivos['IATA'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2])

In [4]:
#Ahora proseguimos a crear las clases

client.command("create class aeropuerto extends V") #Tira error pero sin embargo se creo en el localhost
client.command("create class aerolinea extends V") #Tira error pero sin embargo se creo en el localhost
client.command("create class arrivos extends V") #Tira error pero sin embargo se creo en el localhost

PyOrientCommandException:  - 

In [22]:
#Ahora teniendo ya las clases proseguimos a utilizar el insert para agregar la información 

##Primero a la clase de aeropuerto
for row in range(aeropuerto_info.shape[0]):
    
    client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[row,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[row,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[row,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[row,3]}")

PyOrientCommandException:  - 

In [14]:
# Como se puede observar en el error que hay probablemente sea que OrientDB no soporta carácteres en español

In [16]:
aeropuerto_info.rename(columns={"Ubicación":"Ubicacion","Teléfono":"Telefono"},inplace=True)

In [28]:
for row in range(aeropuerto_info.shape[0]):
    
    client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[row,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[row,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[row,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[row,3]}")

TimeoutError: timed out

In [6]:
aeropuerto_info

,Nombre,Ubicación,Timezone,Teléfono,cod aero
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD
1,San Salvador Cuscatlan Airport (SAL),"RN-5, San Salvador, El Salvador",(UTC/GMT -6),+503 2349 9455,SAL
2,Panama City Albrook Marcos A. Gelabert Airport...,"Avenida Canfield, Panamá, Panama",(UTC/GMT -5),+507 501-9272,PAC
3,Guatemala City La Aurora Airport (GUA),"Guatemala City, Guatemala",(UTC/GMT -6),+502 2321 5000,GUA
4,Puerto Rico Airport (PUR),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,PUR
5,Belize City Municipal Airport (TZA),"Belize City, Belize",(UTC/GMT -6),+501 224-5671,TZA
6,Cap Haitien Cap Haitien Airport (CAP),"Cap-Haitien, Haiti",(UTC/GMT -5),+509 44 80 5107,CAP
7,Montevideo Carrasco Airport (MVD),"101, 14000 Ciudad de la Costa, Uruguay",(UTC/GMT -2),+598 2604 0329,MVD
8,San Juan Luis Munoz Marin Airport (SJU),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,SJU
9,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ


In [5]:
aeropuerto_info['Nombre'] = aeropuerto_info['Nombre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [33]:
client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[0,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[0,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[0,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[0,3]}")

BlockingIOError: [Errno 35] Resource temporarily unavailable

In [39]:
aeropuerto_info = pd.read_csv("DataFrames/Info_Basica_Aeropuertos.csv").iloc[:,1:]
arrivos = pd.read_csv("DataFrames/arrivos.csv").iloc[:,1:]
aerolineas = pd.read_csv("DataFrames/Aerolineas.csv").iloc[:,1:]
aerolineas.fillna("NO",inplace=True)
aerolineas.drop_duplicates(subset=['IATA'],inplace=True)
aerolineas = aerolineas[aerolineas['IATA'].str.len() <= 2]
arrivos = arrivos.applymap(str)
arrivos.drop_duplicates(inplace=True,subset=['Numero de Vuelo'])
arrivos["cod origen"] = arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)
arrivos['cod destino'] = arrivos.apply(lambda x: x['Destino'].split()[-1].replace('(','').replace(')',''), axis=1)
aeropuerto_info['cod aero'] = aeropuerto_info.apply(lambda x: x['Nombre'].split()[-1].replace('(','').replace(')',''), axis=1)
arrivos['cod vuelo'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2]+a[3:])
arrivos['IATA'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2])

In [ ]:
aeropuerto_info.to_csv("Aeropuerto_info_ODB.csv")
aerolineas.to_csv("Aerolineas_ODB.csv")
arrivos.to_csv("Vuelos_ODB.csv")

### Solución al problema

Como se pudo observar anteriormente, OrientDB en Python tiene muchos errores. Por lo cual se decidio implementar Selenium para poder insertar los datos y hacer de esta manera la base de datos de los aeropuertos. 

In [1]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import ui
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [2]:
chromedriver_autoinstaller.install()  # Instala automáticamente la versión correcta de ChromeDriver

'/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/chromedriver_autoinstaller/118/chromedriver'

### Base de datos de Aeropuertos

In [83]:
opt = webdriver.ChromeOptions() 
opt.add_argument("--start-maximized") 

driver = webdriver.Chrome(options=opt)
driver.get("http://localhost:2480/studio/index.html#")


wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "user")))

# Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
username_field = driver.find_element(By.ID, 'user')
username_field.send_keys('root') 

wait.until(EC.presence_of_element_located((By.ID, "password")))

# Localiza el campo de entrada del nombre de password por su 'id' y envía las teclas
password_field = driver.find_element(By.ID, 'password')
password_field.send_keys('root')  

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="database-connect"]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/nav/div/div[2]/ul[1]/li[2]/a[1]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/nav/div/div[2]/ul[1]/li[2]/a[1]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[2]/div[3]/div/div/div[2]/classes-list[1]/div/div[2]/div/button'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="inputName"]')))

nombre = 'Aeropuertos'
name = driver.find_element(By.ID, 'inputName')
name.send_keys(nombre) 

wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[28]/new-vertex-modal/div/div/div[3]/button'))).click()

In [84]:
#Seleccionar el vertex, donde dice 2 es donde se puede cambiar el row
wait = WebDriverWait(driver, 10)

df = aeropuerto_info

wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[2]/div[3]/div/div/div[2]/classes-list[1]/div/div[3]/div/table/tbody/tr[1]/td[3]'))).click()

#Create new record
wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[1]/div/div[2]/div/div[3]/button'))).click()

for column in df.columns:

    wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[1]/div/div/div[2]/button'))).click()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div/div/div[2]/form/div[1]/div/input")))

    field_name = driver.find_element(By.XPATH, '/html/body/div[4]/div/div/div[2]/form/div[1]/div/input')
    field_name.send_keys(column) 

    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div/div[3]/button[2]'))).click()

for rows in range(df.shape[0]):

    for cols in range(1,df.shape[1]+1):

        wait.until(EC.presence_of_element_located((By.XPATH, 
                                                   f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')))

        # Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
        username_field = driver.find_element(By.XPATH, 
                                             f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')
        username_field.send_keys(df.iloc[rows,cols-1]) 

    wait.until(EC.presence_of_element_located((By.XPATH, 
                                               '//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[1]/div/div/div[1]/button'))).click()
    for cols in range(1,df.shape[1]+1):

        wait.until(EC.presence_of_element_located((By.XPATH, 
                                                   f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')))

        # Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
        username_field = driver.find_element(By.XPATH, 
                                             f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')
        username_field.clear()

## Base de datos de Vuelos

In [91]:
opt = webdriver.ChromeOptions() 
opt.add_argument("--start-maximized") 

driver = webdriver.Chrome(options=opt)
driver.get("http://localhost:2480/studio/index.html#")


wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, "user")))

# Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
username_field = driver.find_element(By.ID, 'user')
username_field.send_keys('root') 

wait.until(EC.presence_of_element_located((By.ID, "password")))

# Localiza el campo de entrada del nombre de password por su 'id' y envía las teclas
password_field = driver.find_element(By.ID, 'password')
password_field.send_keys('root')  

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="database-connect"]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/nav/div/div[2]/ul[1]/li[2]/a[1]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[1]/div/nav/div/div[2]/ul[1]/li[2]/a[1]'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[2]/div[3]/div/div/div[2]/classes-list[1]/div/div[2]/div/button'))).click()

wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="inputName"]')))

nombre = 'Vuelos'
name = wait.until(EC.presence_of_element_located((By.ID, 'inputName')))
name.send_keys(nombre) 

wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[29]/new-vertex-modal/div/div/div[3]/button'))).click()

In [ ]:
#Seleccionar el vertex, donde dice 2 es donde se puede cambiar el row
wait = WebDriverWait(driver, 10)

df = arrivos[list(arrivos.columns[1:-1])]

wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[2]/div[3]/div/div/div[2]/classes-list[1]/div/div[3]/div/table/tbody/tr[1]/td[3]'))).click()

#Create new record
wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/div/div/div[1]/div/div[2]/div/div[3]/button'))).click()

for column in df.columns:

    wait.until(EC.presence_of_element_located((By.XPATH, 
                                           '//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[1]/div/div/div[2]/button'))).click()
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div/div/div[2]/form/div[1]/div/input")))

    field_name = driver.find_element(By.XPATH, '/html/body/div[4]/div/div/div[2]/form/div[1]/div/input')
    field_name.send_keys(column) 

    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div/div[3]/button[2]'))).click()

for rows in range(df.shape[0]):

    for cols in range(1,df.shape[1]+1):

        wait.until(EC.presence_of_element_located((By.XPATH, 
                                                   f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')))

        # Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
        username_field = driver.find_element(By.XPATH, 
                                             f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')
        username_field.send_keys(df.iloc[rows,cols-1]) 

    wait.until(EC.presence_of_element_located((By.XPATH, 
                                               '//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[1]/div/div/div[1]/button'))).click()
    for cols in range(1,df.shape[1]+1):

        wait.until(EC.presence_of_element_located((By.XPATH, 
                                                   f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')))

        # Localiza el campo de entrada del nombre de usuario por su 'id' y envía las teclas
        username_field = driver.find_element(By.XPATH, 
                                             f'//*[@id="wrap"]/div[7]/div[2]/div/ng-include/div/div[2]/ng-include/div/div[2]/div/form/fieldset/div/div/ng-form/div[{cols}]/div/div/ng-include/textarea')
        username_field.clear()

In [80]:
df.shape[1]

5

In [90]:
arrivos

,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo,IATA
0,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,SDQ,5X402,5X
1,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,SDQ,AV208,AV
2,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,SDQ,CM107,CM
3,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,SDQ,TK9551,TK
4,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,SDQ,KL3056,KL
...,...,...,...,...,...,...,...,...,...
15922,Orlando [MCO],Miami Airport (MIA),IB 4728,9:45,nan,MCO,MIA,IB4728,IB
15923,San José [SJO],Miami Airport (MIA),AA 1204,6:00,10:28,SJO,MIA,AA1204,AA
15924,San José [SJO],Miami Airport (MIA),IB 4551,6:00,10:28,SJO,MIA,IB4551,IB
15925,Detroit [DTW],Miami Airport (MIA),DL 1468,8:00,10:40,DTW,MIA,DL1468,DL


In [102]:
df.shape[1]

9

In [115]:
for cols in range(1,df.shape[1]+1):
    
    print(cols)

1
2
3
4
5
6
7
8
9


In [121]:
arrivos[list(arrivos.columns[1:-1])]

,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo
0,Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,SDQ,5X402
1,Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,SDQ,AV208
2,Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,SDQ,CM107
3,Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,SDQ,TK9551
4,Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,SDQ,KL3056
...,...,...,...,...,...,...,...
15922,Miami Airport (MIA),IB 4728,9:45,nan,MCO,MIA,IB4728
15923,Miami Airport (MIA),AA 1204,6:00,10:28,SJO,MIA,AA1204
15924,Miami Airport (MIA),IB 4551,6:00,10:28,SJO,MIA,IB4551
15925,Miami Airport (MIA),DL 1468,8:00,10:40,DTW,MIA,DL1468
